# TODO
- Extreme amount extraction.
- True income vs loaned money display.
- Multi indexing on year, month, week, date.
- Code refactoring.

In [1]:
%matplotlib inline

In [2]:
from IPython.display import display
from IPython.display import clear_output
from ipywidgets import *
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import locale

# Settings

In [3]:
locale.setlocale(locale.LC_ALL, 'en_GB.utf8');

# Display functions

In [4]:
def amount_to_float(x):
    return float(x) / 100.

def amount_to_str(x):
    x = amount_to_float(x)
    return locale.format("%.2f", x, grouping=True)

# Data

In [5]:
df = pd.read_csv('afschriften.csv')

In [6]:
cats = pd.read_csv('categories.csv')

# Preprocessing
###### Column renaming

In [7]:
renamings = {
    'Datum': 'date',
    'Naam / Omschrijving': 'name',
    'Rekening': 'account_from',
    'Tegenrekening': 'account_to',
    'Code': 'code',
    'Af Bij': 'sign',
    'Bedrag (EUR)': 'amount',
    'MutatieSoort': 'device',
    'Mededelingen': 'note',
}
df = df.rename(columns=renamings)

###### Date parsing

In [8]:
# Parse the date column to datetimes and set as index.
df.date = pd.to_datetime(df.date, format='%d%m%Y')

TypeError: 'int' object is unsliceable

###### Account to parsing

In [ ]:
df.account_to = df.account_to.replace(np.nan, '-')

###### Sign and amount parsing

In [ ]:
# Transform the sign column to symbols.
df.sign = df.sign.apply(lambda sign: '+' if sign == 'Bij' else '-')

In [ ]:
# Transform amount column from str to float.
df.amount = df.amount.replace({',': '.'}, regex=True).astype(np.float64)

In [ ]:
# Transform amount column from float to int, without losing the cents.
df.amount = df.amount.apply(lambda x: x * 100).astype(int)

In [ ]:
# Apply the symbol in the sign column to the amounts.
f = lambda x: x.amount if x.sign == '+' else -x.amount
df.amount = df[['sign', 'amount']].apply(f, axis=1, broadcast=True)
df.amount = df.amount.astype(int)

## Data generation

In [ ]:
df['week'] = df.date.apply(lambda x:x.week)

In [ ]:
df['month'] = df.date.apply(lambda x:x.month)

In [ ]:
df['year'] = df.date.apply(lambda x:x.year)

###### Finished

In [ ]:
display(df.head())

# Categorisation
## Categories

In [ ]:
def create_category_overview(df, cats):
    df = df.loc[:, ('account_to', 'amount')]
    df = pd.merge(cats, df, on='account_to').drop('account_to', axis=1)
    df = df.groupby('category').sum()
    return df

def print_category_overview(overview):
    overview = overview.sort_values(by='amount')
    overview.amount = overview.amount.apply(amount_to_str)
    display(overview)

## Uncategorised

In [ ]:
def print_uncategorised(df):
    idxs = ~df.account_to.isin(cats.account_to), ('name', 'account_to', 'amount')
    display(df.loc[idxs])

# Overview: income & expense

In [ ]:
def describe(s):
    """Creates a custom description of a series.
    """
    return {
        'total': s.sum(),
        'positive': s.loc[s >= 0].sum(),
        'negative': s.loc[s < 0].sum(),
        'min': s.min(),
        'max': s.max(),
        'avg': s.mean(),
        'median': s.median(),
#         'mode': s.mode(),
        'count': s.count(),
    }

# TODO: fix formatting with plus/minus signs.
def print_description(d, name):
    """Prints a description. 
    
    Naming is based on the assumption that
    the description describes a revenue stream.
    """
    # All amounts to str representation.
    d = {k: v if k == 'count' else amount_to_str(v) for k, v in d.items()}
    
    print('--- {} ---'.format(name))
    print('expense {}'.format(d['negative']))
    print('revenue  {}'.format(d['positive']))
    print('income  {}'.format(d['total']))
    print()
    print('count    {}'.format(d['count']))
    print('min     {}'.format(d['min']))
    print('max      {}'.format(d['max']))
    print('avg     {}'.format(d['avg']))
    print('median  {}'.format(d['median']))
#     print('mode    {:.0f}'.format(d['mode']))

In [ ]:
def summed_mutation_overview(period):
    """Sums all mutations per account_to and adds name to each account.
    
    df: Entire history in a DataFrame.
    start: DateTime for start (inclusive) of the period.
    end: DateTime for end (inclusive) of the period.
    Returns: DataFrame with columns 'account_to, name, amount'. Amounts
        are summed by account_to. The first name found for an account_to
        in the df is assigned as a value in the respective name column. 
        The only exception to this is the account_to with value '-', this
        gets the name '-'.
    """
    # Combine amounts with respect to the account_to and add a name.
    accountto_amount = period[['account_to', 'amount']].groupby('account_to', as_index=False).sum()
    accountto_name = period[['account_to', 'name']].groupby('account_to', as_index=False).first()
    overview = pd.merge(accountto_name, accountto_amount, how='left', on='account_to')
    # Assigning name '-' to account_to == '-'.
    overview.loc[overview.account_to == '-', 'name'] = '-'
    # Prettify and sort.
    overview = overview.sort_values(by=['amount'])
    overview.amount = overview.amount.apply(amount_to_str)
    return overview

# Period selector
Select a period by adjusting the sliders, then select the cell below the sliders and run "Run all below".

In [ ]:
def get_period(df,
               start=dt.date.today().replace(year=1900),
               end=dt.date.today()):
    period = df.loc[df.date >= start]
    return period.loc[period.date <= end]

In [ ]:
dates = df.date
start = dates.iloc[-1]
end = dates.iloc[0]
drange = pd.date_range(start, end)
drange = [pd.to_datetime(str(i)).strftime('%d.%m.%Y') for i in drange.values]

start_slider = widgets.SelectionSlider(
    options=drange,
    value=drange[0],
    description='Start:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
)

end_slider = widgets.SelectionSlider(
    options=drange,
    value=drange[-1],
    description='End:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
)

display(start_slider, end_slider)

In [ ]:
def current_month_start():
    start = dt.date.today()
    if start.day > 25:
        start += dt.timedelta(7)
    return start.replace(day=1)

use_slider = True
if use_slider:
#     period = get_period(df, start_slider.value, end_slider.value)
    period = get_period(df, pd.Timestamp('2017-03-01'), pd.Timestamp('2017-04-30'))
else:
    period = get_period(df, current_month_start())

## Tables and graphs

### Description

In [ ]:
print_description(describe(period.amount), "TOTAL")

### Account mutations

In [ ]:
summed_mutation_overview(period)

### Category mutations

In [ ]:
print_category_overview(create_category_overview(period, cats))

#### Uncategorised

In [ ]:
print_uncategorised(period)

### Balance

In [ ]:
period = period.sort_values('date', ascending=True)
period['balance'] = period.amount.cumsum()

In [ ]:
values = period.loc[:, ['date', 'amount']].groupby('date', as_index=True).sum()
values['balance'] = values.amount.cumsum()
values.balance.map(amount_to_float).plot(kind='line');

In [ ]:
values = period.loc[:, ['date', 'amount']].groupby('date', as_index=True).sum()
values.amount.map(amount_to_float).plot(kind='bar');

##### Heatmaps

In [ ]:
pt_ym_amount = period[['year','month','amount']].groupby(['year','month'], as_index=False).sum()
pt_ym_amount.amount = pt_ym_amount.amount.map(amount_to_float)
pt_ym_amount = pt_ym_amount.pivot('year','month','amount')

In [ ]:
cmap = sns.diverging_palette(20, 220, sep=20, as_cmap=True)
fig, ax = plt.subplots(figsize=(12,5)) 
sns.heatmap(pt_ym_amount, linewidth=.4, fmt='.0f', annot=True, cmap=cmap, ax=ax);

In [ ]:
pt_ym_balance = period[['year','month','balance']].groupby(['year','month'], as_index=False).sum()
pt_ym_balance.balance = pt_ym_balance.balance.map(amount_to_float)
pt_ym_balance = pt_ym_balance.pivot('year','month','balance')

In [ ]:
cmap = sns.diverging_palette(20, 220, sep=20, as_cmap=True)
fig, ax = plt.subplots(figsize=(12,5)) 
sns.heatmap(pt_ym_balance, linewidth=.4, fmt='.0f', annot=True, cmap=cmap, ax=ax);

# Reports and reasoning

### (Normal) spending/income

In [ ]:
def display_bounded_range(df, lbound, rbound):
    mask = df.amount.between(lbound, rbound)
    amounts = df.amount.loc[mask]
    
    perc = 100.0 * (float(amounts.count()) / float(df.amount.count()))
    print('Percentage {:.2f}%'.format(perc))
    
    amounts = amounts.map(amount_to_float)
    sns.distplot(amounts, hist=False, rug=True)
    return amounts.map(amount_to_str)

## Std based

In [ ]:
records = df.loc[df.sign == '+']
records.amount = records.amount.map(abs)
records = records.sort_values('amount')
std = records.amount.std()
std

In [ ]:
values = display_bounded_range(records, 0, std)

In [ ]:
values = display_bounded_range(records, 1*std, 2*std)

In [ ]:
values = display_bounded_range(records, 0, 2*std)

In [ ]:
values = display_bounded_range(records, 2*std, 3*std)

In [ ]:
values = display_bounded_range(records, 0, 3*std)

In [ ]:
values = display_bounded_range(records, 3*std, np.inf)

In [ ]:
values = display_bounded_range(records, 0, np.inf)

## Quantile based

In [ ]:
rbound = records.amount.quantile(.75)
values = display_bounded_range(records, 0, rbound)

In [ ]:
rbound = records.amount.quantile(.95)
values = display_bounded_range(records, 0, rbound)

In [ ]:
rbound = records.amount.quantile(.99)
values = display_bounded_range(records, 0, rbound)